In [ ]:
# default_exp benchmarking

# benchmarking

> This module contains new evaluation protocol for UBC Phototour local patch dataset

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import numpy as np
import gc
from fastprogress.fastprogress import progress_bar
from scipy.spatial.distance import cdist
from sklearn.metrics.pairwise import paired_distances
from sklearn.metrics import average_precision_score
def evaluate_mAP_snn_based(descriptors, labels, img_labels):
    '''Function to calculate mean average precision, over per-image based matching using Lowe SNN ratio'''
    APs = []
    unique_img_labels = sorted(np.unique(img_labels))
    for img_idx in progress_bar(unique_img_labels):
        current_batch = img_labels == img_idx
        cur_descs = descriptors[current_batch]
        cur_labels = labels[current_batch]
        NN = cur_labels.shape[0]
        pos_labels_repeat = np.broadcast_to(cur_labels.reshape(1,-1),(NN,NN))
        pos_mask = (pos_labels_repeat == pos_labels_repeat.T)
        pos_mask_not_anchor = pos_mask != np.eye(NN, dtype=np.bool)
        neg_idx = np.zeros((NN), dtype=np.int32)
        if NN > 1000: # To avoid OOM, we will find hard negative in batches
            bs1 = 128
            nb = (NN // bs1)  
            for i in range(nb):
                st = i*bs1
                fin = min(NN, (i+1)*bs1)
                if fin == st:
                    break
                dm = cdist(cur_descs[st:fin], cur_descs) +\
                        1000.0 * pos_mask[st:fin] + \
                        1000.0 * np.eye(NN, dtype=np.bool)[st:fin]
                min_neg_idxs = np.argmin(dm, axis=1)
                neg_idx[st:fin] = min_neg_idxs
        # We want to create all possible anchor-positive combinations
        pos_idxs = np.broadcast_to(np.arange(NN).reshape(1,-1),(NN,NN))[pos_mask_not_anchor]
        anc_idxs = np.nonzero(pos_mask_not_anchor)[0]
        pos_mask = None
        neg_idxs = neg_idx[anc_idxs]
        pos_dists = paired_distances(cur_descs[anc_idxs], cur_descs[pos_idxs])
        neg_dists = paired_distances(cur_descs[anc_idxs], cur_descs[neg_idxs])
        correct = pos_dists <= neg_dists
        snn = np.minimum(pos_dists,neg_dists) / np.maximum(pos_dists,neg_dists)
        snn[np.isnan(snn)] = 1.0
        ap = average_precision_score(correct, 1-snn)
        APs.append(ap)
        pos_mask = None
        pos_mask_not_anchor = None
        cur_descs = None
        pos_labels_repeat = None
        dm = None
        gc.collect()
    return np.array(APs).mean()